In [40]:
import tensorflow as tf
import tensorflow_hub as hub
from bert import tokenization, optimization, run_classifier
import os
import csv
from datetime import datetime

In [44]:
def create_model(is_training, input_ids, input_mask, segment_ids, labels,
                 num_labels, bert_hub_module_handle):
    """
    Creates a classification model
    """
    tags = set()
    if is_training:
        tags.add("train")
    bert_module = hub.Module(bert_hub_module_handle,
                            tags=tags,
                            trainable=True)
    bert_inputs = dict(input_ids=input_ids,
                       input_mask=input_mask,
                       segment_ids=segment_ids)
    bert_outputs = bert_module(inputs=bert_inputs,
                               signature="tokens",
                               as_dict=True)
    output_layer = bert_outputs["pooled_output"]
    hidden_size = output_layer.shape[-1].value
    output_weight = tf.get_variable(
        "output_weights", [num_labels, hidden_size],
        initializer=tf.truncated_normal_initializer(stddev=0.02))
    output_bias = tf.get_variable("output_bias", [num_labels],
                                  initializer=tf.zeros_initializer())
    with tf.variable_scope("loss"):
        if is_training:
            output_layer = tf.nn.dropout(output_layer, keep_prob=0.9)
        logits = tf.matmul(output_layer, output_weight, transpose_b=True)
        logits = tf.nn.bias_add(logits, output_bias)
        probabilities = tf.nn.softmax(logits, axis=-1)
        log_probs = tf.nn.log_softmax(logits, axis=-1)
        one_hot_labels = tf.one_hot(labels, depth=num_labels, dtype=tf.float32)
        predicted_labels = tf.squeeze(
            tf.argmax(log_probs, axis=-1, output_type=tf.int32))
        per_example_loss = -tf.reduce_sum(one_hot_labels * log_probs, axis=-1)
        loss = tf.reduce_mean(per_example_loss)
        return (loss, per_example_loss, logits, probabilities, log_probs,
                predicted_labels)

In [35]:
def model_fn_build(num_labels, learning_rate, num_train_steps,
                   num_warmup_steps, bert_hub_module_handle):
    def model_fn(features, labels, mode, params):
        tf.logging.info("****** Features ******")
        for name in sorted(features.keys()):
            tf.logging.info("name = %s, shape = %s" %
                            (name, features[name].shape))
        input_ids = features["input_ids"]
        input_mask = features["input_mask"]
        segment_ids = features["segment_ids"]
        label_ids = features["label_ids"]
        is_training = (mode == tf.estimator.ModeKeys.TRAIN)
        (total_loss, per_example_loss, logits, probabilities, log_probs,
         predicted_labels) = create_model(is_training, input_ids, input_mask,
                                          segment_ids, label_ids, num_labels,
                                          bert_hub_module_handle)
        output_spec = None
        if mode == tf.estimator.ModeKeys.TRAIN:
            train_op = optimization.create_optimizer(total_loss,
                                                     learning_rate,
                                                     num_train_steps,
                                                     num_warmup_steps,
                                                     use_tpu=False)
            output_spec = tf.estimator.EstimatorSpec(mode=mode,
                                                     loss=total_loss,
                                                     train_op=train_op)
        elif mode == tf.estimator.ModeKeys.EVAL:

            def metric_fn(per_example_loss, label_ids, logits):
                predictions = tf.argmax(logits, axis=-1, output_type=tf.int32)
                accuracy = tf.metrics.accuracy(labels=label_ids,
                                               predictions=predictions)
                loss = tf.metrics.mean(per_example_loss)
                return {
                    "eval_accuracy": accuracy,
                    "eval_loss": loss,
                }

            eval_metrics = metric_fn(per_example_loss, label_ids, logits)
            output_spec = tf.estimator.EstimatorSpec(
                mode=mode, loss=total_loss, eval_metric_ops=eval_metrics)
        elif mode == tf.estimator.ModeKeys.PREDICT:
            output_spec = tf.estimator.EstimatorSpec(mode=mode,
                                                     predictions={
                                                         "probabilities":
                                                         probabilities,
                                                         "log_probs":
                                                         log_probs,
                                                         "predicted_labels":
                                                         predicted_labels
                                                     })
        else:
            raise ValueError(
                "Only TRAIN, EVAL and PREDICT modes are supported: %s" %
                (mode))
        return output_spec

    return model_fn

In [5]:
def bbc_data_processor(data_dir, set_type, test_label):
    file_path = os.path.join(data_dir, "bbc_" + set_type + ".csv")
    with tf.gfile.Open(file_path, "r") as f:
        reader = csv.reader(f, delimiter=",", quotechar=None)
        examples = []
        labels = []
        labels_test = []
        for (i, line) in enumerate(reader):
            if i == 0:
                continue
            guid = set_type + "-" + str(i)
            text_a = tokenization.convert_to_unicode(line[1])
            label = tokenization.convert_to_unicode(line[0])
            labels.append(label)
            if set_type == "test":
                label = test_label
            labels_test.append(label)
            examples.append(
                run_classifier.InputExample(guid=guid,
                                            text_a=text_a,
                                            text_b=None,
                                            label=label))
        return examples, labels, labels_test

In [6]:
DATA_DIR = "./data"

In [7]:
train_InputExamples, train_labels, train_labels_test = bbc_data_processor(data_dir=DATA_DIR,set_type="train",test_label="tech")

In [8]:
train_labels_list = list(set(train_labels))

In [10]:
test_InputExamples,test_labels, test_labels_test = bbc_data_processor(data_dir=DATA_DIR,set_type="test",test_label="tech")

In [11]:
test_labels_list = list(set(test_labels))

In [12]:
BERT_MODEL_HUB = "https://tfhub.dev/google/bert_uncased_L-12_H-768_A-12/1"

In [13]:
def create_tokenizer_from_hub_module(bert_hub_module_handle):
    """Get the vocab file and casing info from the Hub module."""
    with tf.Graph().as_default():
        bert_module = hub.Module(bert_hub_module_handle)
        tokenization_info = bert_module(signature="tokenization_info",
                                        as_dict=True)
        with tf.Session() as sess:
            vocab_file, do_lower_case = sess.run([
                tokenization_info["vocab_file"],
                tokenization_info["do_lower_case"]
            ])

    return tokenization.FullTokenizer(vocab_file=vocab_file,
                                           do_lower_case=do_lower_case)

In [14]:
tokenizer = create_tokenizer_from_hub_module(BERT_MODEL_HUB)

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


In [15]:
tokenizer.tokenize("This here's an example of using the BERT tokenizer")

['this',
 'here',
 "'",
 's',
 'an',
 'example',
 'of',
 'using',
 'the',
 'bert',
 'token',
 '##izer']

In [17]:
MAX_SEQ_LENGTH = 128

In [18]:
train_features = run_classifier.convert_examples_to_features(train_InputExamples, train_labels_list, MAX_SEQ_LENGTH, tokenizer)

INFO:tensorflow:Writing example 0 of 1780


INFO:tensorflow:Writing example 0 of 1780


INFO:tensorflow:*** Example ***


INFO:tensorflow:*** Example ***


INFO:tensorflow:guid: train-1


...

INFO:tensorflow:input_mask: 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1


INFO:tensorflow:segment_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0


INFO:tensorflow:segment_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0


INFO:tensorflow:label: entertainment (id = 1)


INFO:tensorflow:label: entertainment (id = 1)


In [19]:
len(train_features)

1780

In [22]:
test_features = run_classifier.convert_examples_to_features(test_InputExamples, test_labels_list, MAX_SEQ_LENGTH, tokenizer)

INFO:tensorflow:Writing example 0 of 445


INFO:tensorflow:Writing example 0 of 445


INFO:tensorflow:*** Example ***


INFO:tensorflow:*** Example ***


INFO:tensorflow:guid: test-1


...

INFO:tensorflow:input_mask: 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1


INFO:tensorflow:segment_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0


INFO:tensorflow:segment_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0


INFO:tensorflow:label: tech (id = 0)


INFO:tensorflow:label: tech (id = 0)


In [23]:
len(test_features)

445

In [24]:
# Compute train and warmup steps from batch size
# These hyperparameters are copied from this colab notebook (https://colab.sandbox.google.com/github/tensorflow/tpu/blob/master/tools/colab/bert_finetuning_with_cloud_tpus.ipynb)
BATCH_SIZE = 32
LEARNING_RATE = 2e-5
NUM_TRAIN_EPOCHS = 10.0
# Warmup is a period of time where hte learning rate 
# is small and gradually increases--usually helps training.
WARMUP_PROPORTION = 0.1
# Model configs
SAVE_CHECKPOINTS_STEPS = 100
SAVE_SUMMARY_STEPS = 10

In [25]:
num_train_steps = int(len(train_features) / BATCH_SIZE * NUM_TRAIN_EPOCHS)

In [26]:
num_train_steps

556

In [31]:
num_warmup_steps = int(num_train_steps * WARMUP_PROPORTION)

In [32]:
num_warmup_steps

55

In [29]:
OUTPUT_DIR = "./output_based_on_private_model/bbc"

In [30]:
run_config = tf.estimator.RunConfig(
    model_dir=OUTPUT_DIR,
    save_summary_steps=SAVE_SUMMARY_STEPS,
    save_checkpoints_steps=SAVE_CHECKPOINTS_STEPS)

In [37]:
model_fn = model_fn_build(num_labels=len(train_labels),
                          learning_rate=LEARNING_RATE,
                          num_train_steps=num_train_steps,
                          num_warmup_steps=num_warmup_steps,
                          bert_hub_module_handle=BERT_MODEL_HUB)

In [38]:
estimator = tf.estimator.Estimator(model_fn=model_fn,
                                   config=run_config,
                                   params={"batch_size": BATCH_SIZE})

INFO:tensorflow:Using config: {'_tf_random_seed': None, '_model_dir': './output_based_on_private_model/bbc', '_experimental_distribute': None, '_is_chief': True, '_num_ps_replicas': 0, '_keep_checkpoint_every_n_hours': 10000, '_experimental_max_worker_delay_secs': None, '_save_checkpoints_steps': 100, '_task_type': 'worker', '_global_id_in_cluster': 0, '_service': None, '_task_id': 0, '_eval_distribute': None, '_save_checkpoints_secs': None, '_save_summary_steps': 10, '_evaluation_master': '', '_device_fn': None, '_keep_checkpoint_max': 5, '_num_worker_replicas': 1, '_train_distribute': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f5370515710>, '_protocol': None, '_master': '', '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_log_step_count_steps': 100}


INFO:tensorflow:Using config: {'_tf_random_seed': None, '_model_dir': './output_based_on_private_model/bbc', '_experimental_distribute': None, '_is_chief': True, '_num_ps_replicas': 0, '_keep_checkpoint_every_n_hours': 10000, '_experimental_max_worker_delay_secs': None, '_save_checkpoints_steps': 100, '_task_type': 'worker', '_global_id_in_cluster': 0, '_service': None, '_task_id': 0, '_eval_distribute': None, '_save_checkpoints_secs': None, '_save_summary_steps': 10, '_evaluation_master': '', '_device_fn': None, '_keep_checkpoint_max': 5, '_num_worker_replicas': 1, '_train_distribute': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f5370515710>, '_protocol': None, '_master': '', '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_log_step_count_steps': 100}


In [39]:
train_input_fn = run_classifier.input_fn_builder(features=train_features,
                                                 seq_length=MAX_SEQ_LENGTH,
                                                 is_training=True,
                                                 drop_remainder=False)

In [45]:
print('Beginning Training!')
current_time = datetime.now()
estimator.train(input_fn=train_input_fn, max_steps=num_train_steps)
print("Training took time ", datetime.now() - current_time)

Beginning Training!
INFO:tensorflow:Calling model_fn.


INFO:tensorflow:Calling model_fn.


INFO:tensorflow:****** Features ******


INFO:tensorflow:****** Features ******


INFO:tensorflow:name = input_ids, shape = (?, 128)


...

INFO:tensorflow:Saving checkpoints for 556 into ./output_based_on_private_model/bbc/model.ckpt.


INFO:tensorflow:Saving checkpoints for 556 into ./output_based_on_private_model/bbc/model.ckpt.


INFO:tensorflow:Loss for final step: 0.004494274.


INFO:tensorflow:Loss for final step: 0.004494274.


Training took time  1:37:40.456834


In [46]:
test_input_fn = run_classifier.input_fn_builder(features=test_features,
                                                seq_length=MAX_SEQ_LENGTH,
                                                is_training=False,
                                                drop_remainder=False)

In [47]:
estimator.evaluate(input_fn=test_input_fn, steps=None)

INFO:tensorflow:Calling model_fn.


INFO:tensorflow:Calling model_fn.


INFO:tensorflow:****** Features ******


INFO:tensorflow:****** Features ******


INFO:tensorflow:name = input_ids, shape = (?, 128)


...

INFO:tensorflow:Saving dict for global step 556: eval_accuracy = 0.2, eval_loss = 6.877123, global_step = 556, loss = 6.8728127


INFO:tensorflow:Saving dict for global step 556: eval_accuracy = 0.2, eval_loss = 6.877123, global_step = 556, loss = 6.8728127


INFO:tensorflow:Saving 'checkpoint_path' summary for global step 556: ./output_based_on_private_model/bbc/model.ckpt-556


INFO:tensorflow:Saving 'checkpoint_path' summary for global step 556: ./output_based_on_private_model/bbc/model.ckpt-556


{'eval_accuracy': 0.2,
 'eval_loss': 6.877123,
 'global_step': 556,
 'loss': 6.8728127}

In [48]:
train_labels_list

['tech', 'entertainment', 'sport', 'business', 'politics']

In [55]:
def getPrediction(in_sentences):
    label_list = ['tech', 'entertainment', 'sport', 'business', 'politics']
    input_examples = [
        run_classifier.InputExample(guid="",
                                    text_a=x,
                                    text_b=None,
                                    label=label_list[0]) for x in in_sentences
    ]
    input_features = run_classifier.convert_examples_to_features(
        input_examples, label_list, MAX_SEQ_LENGTH, tokenizer)
    predict_input_fn = run_classifier.input_fn_builder(
        features=input_features,
        seq_length=MAX_SEQ_LENGTH,
        is_training=False,
        drop_remainder=False)
    predictions = estimator.predict(predict_input_fn)
    return [(sentence, prediction['probabilities'], prediction['log_probs'],
             label_list[prediction['predicted_labels']])
            for sentence, prediction in zip(in_sentences, predictions)]

In [52]:
pred_sentences = [
    "hobbit picture  four years away  lord of the rings director peter jackson has said that it will be up to four years before he starts work on a film version of the hobbit.  the oscar winner said on a visit to sydney there was a  desire  to make it  but not before lengthy negotiations.  i think it s gonna be a lot of lawyers sitting in a room trying to thrash out a deal before it will ever happen   said the new zealander. the rights to jrr tolkien s book are split between two major film studios. jackson  who is currently filming a remake of hollywood classic king kong  said he thought that the sale of mgm studios to the sony corporation would cast further uncertainty on the project. the 43-year-old was in the australian city to visit a lord of the rings exhibition  which has attracted 140 000 visitors since it opened in december.  the film-maker recently sued film company new line cinema for undisclosed damages over alleged withheld profits and lost revenue from the first part of the middle earth trilogy. the fellowship of the ring from 2001 went on to make worldwide profits of $291 million (£152 million). jackson is thought to have secured the most lucrative film directing deal in history to remake king kong  which is currently in production in wellington. the picture  which stars naomi watts and oscar winner adrien brody  is due to be released in december. jackson has also committed to making a film version of lovely bones  based on the best-selling book by alice sebold.",
    "game firm holds  cast  auditions video game firm bioware is to hold open auditions for people to become cast members for future games.  the company  which makes role playing games such as knights of the old republic and neverwinter nights  is seeking people aged 18 to 99. the canada-based company says it was looking for  a wide variety of people to use as face models for characters . everyone chosen to appear in a video game will receive a performer s fee for the use of their image. the company is inviting people to come along to a shopping mall in west edmonton  alberta  on friday and saturday  bringing along a piece of photo identification.   there are hundreds and hundreds of characters in a typical bioware game   said shauna perry  bioware s audio and external resources producer.  those people live in any city and village and so we need ordinary people  people with interesting faces.  she added:  not everyone is a model in the world so we don t want just models in our games.  people chosen to appear in a game will have their head scanned in three dimensions. hundreds of photos of the person s head are taken so that a model of the head can be generated in 3d.  the 3d model will look exactly like the person - it s really quite incredible how detailed they are   said ms perry. she said chosen participants will have no control over how the image is used in a computer game.  we cannot give people any control over how the images are used.  but their face could be used in multiple games - so they could be the hero in one  the villain in another and just a merchant in a third.",
    "clarke plans migrant point scheme anyone planning to move to the uk will have to pass a test to prove they can contribute to the country  home secretary charles clarke has said.  he is proposing a points system similar to australia s but would avoid the quota system planned by the tories. mr clarke  who will unveil his plans on monday  said economic migration helped the uk but  needed proper policing . the lib dems say they will look at his plans  but tory liam fox said his party offered a  clear choice  on the issue. the conservative party co-chairman said the british electorate had a choice between a labour government that had  done nothing for eight years and will not set a limit  on immigration and a tory one that would impose quotas.  the home secretary said  by 2008  he wanted everyone given a visa and entering the uk to have their fingerprints taken  to  ensure we can know everybody who is in the country . speaking on bbc one s breakfast with frost  he said  economic migrants are of great value to this country   but stressed that proper policing was needed to ensure that they do not become a  burden on society . he said:  we will establish a system ... which looks at the skills  talents and abilities of people seeking to come and work in this country  and ensures that when they come here they have a job and can contribute to the economy of the country.   the home secretary  whose five-year blueprint for immigration and asylum is expected to be published on monday  also rejected claims that the immigration debate encouraged bigotry.  the issue of who does come into this country  and whether they are entitled to be in this country  who does settle here  how we have border controls  is a perfectly legitimate aspect of public debate   he said. liberal democrat home affairs spokesman mark oaten said:  whilst it is good that labour has rejected the tory idea of quotas on asylum  the jury is still out on the home office s ability to deliver a fair and efficient asylum system.   mr howard has said britain should take its fair share of the world s  genuine refugees . but he claims the current asylum system is being abused - and with it britain s generosity. trevor phillips  chairman of the commission for racial equality  called on mr clarke to denounce the suggestion britain s hospitality was being tested by immigration.  tell that to the 44 000 doctors in the nhs and the 70 000 nurses without whom we would really see what pressure on the health service means   he said.  ditto the teachers  from south africa  australia  jamaica  who are reducing the sizes of our classes and schools.  the refugee council said mr howard s proposals would mean there would be no safe haven in the uk.",
    "radcliffe will compete in london paula radcliffe will compete in the flora london marathon this year after deciding her schedule for 2005.  the 31-year-old won the race in 2002 on her marathon debut  defended her title 12 months later and will now seek a third title in the 17 april race.  it doesn t get any better than this for the 25th anniversary   said race director david bedford.  after announcing the greatest men s field ever we now have the greatest women s distance runner ever.  three years ago radcliffe smashed the women s world record in two hours 18 minutes 15 seconds.  the bedford star returned to london 12 months later  lowering her mixed-race world record of 2:17:18  which she set in chicago in october 2003  by one minute 53 secs. radcliffe s career took a setback when she failed to complete the olympic marathon and later dropped out of the athens 10 000m last august. but the 31-year-old bounced back to win the new york marathon in november. radcliffe  however  passed up the chance to go for the  big city  marathon grand slam. with wins in chicago  london and new york  only the boston marathon remains to be conquered but that takes place a day after london.  boston is definitely a race i want to do at some point  but london is very special to me   said radcliffe.  i don t pick races thinking about things like pressure. i pick the ones in my heart i really want to do.  i love the atmosphere  crowds and course and know it will always be a great quality race.  it is also the 25th anniversary this year which adds to the occasion."
]

In [53]:
len(pred_sentences)

4

In [56]:
predictions = getPrediction(pred_sentences)

INFO:tensorflow:Writing example 0 of 4


INFO:tensorflow:Writing example 0 of 4


INFO:tensorflow:*** Example ***


INFO:tensorflow:*** Example ***


INFO:tensorflow:guid: 


...

INFO:tensorflow:Restoring parameters from ./output_based_on_private_model/bbc/model.ckpt-556


INFO:tensorflow:Running local_init_op.


INFO:tensorflow:Running local_init_op.


INFO:tensorflow:Done running local_init_op.


INFO:tensorflow:Done running local_init_op.


In [57]:
predictions

[('hobbit picture  four years away  lord of the rings director peter jackson has said that it will be up to four years before he starts work on a film version of the hobbit.  the oscar winner said on a visit to sydney there was a  desire  to make it  but not before lengthy negotiations.  i think it s gonna be a lot of lawyers sitting in a room trying to thrash out a deal before it will ever happen   said the new zealander. the rights to jrr tolkien s book are split between two major film studios. jackson  who is currently filming a remake of hollywood classic king kong  said he thought that the sale of mgm studios to the sony corporation would cast further uncertainty on the project. the 43-year-old was in the australian city to visit a lord of the rings exhibition  which has attracted 140 000 visitors since it opened in december.  the film-maker recently sued film company new line cinema for undisclosed damages over alleged withheld profits and lost revenue from the first part of the 

In [65]:
def list_from_csv(file_path):
    sentences_list = []
    label_list = []
    with tf.gfile.Open(file_path, "r") as f:
        reader = csv.reader(f, delimiter=",", quotechar=None)
        for (i, line) in enumerate(reader):
            if i == 0:
                continue
            label_list.append(line[0])
            sentences_list.append(line[1])
    return label_list, sentences_list

In [66]:
file_path = "./data/bbc_test.csv"

In [67]:
label_list, sentences_list = list_from_csv(file_path)

In [68]:
len(sentences_list)

445

In [69]:
len(label_list)

445

In [70]:
label_list

['entertainment',
 'tech',
 'politics',
 'sport',
 'sport',
 'tech',
 'business',
 'tech',
 'entertainment',
 'entertainment',
 'politics',
 'politics',
 'business',
 'entertainment',
 'sport',
 'entertainment',
 'entertainment',
 'business',
 'sport',
 'politics',
 'tech',
 'business',
 'sport',
 'entertainment',
 'tech',
 'politics',
 'politics',
 'sport',
 'sport',
 'business',
 'business',
 'business',
 'business',
 'business',
 'entertainment',
 'business',
 'politics',
 'politics',
 'tech',
 'tech',
 'entertainment',
 'politics',
 'entertainment',
 'business',
 'politics',
 'sport',
 'sport',
 'business',
 'tech',
 'business',
 'tech',
 'sport',
 'business',
 'business',
 'politics',
 'sport',
 'sport',
 'politics',
 'politics',
 'entertainment',
 'entertainment',
 'politics',
 'business',
 'politics',
 'politics',
 'business',
 'tech',
 'politics',
 'politics',
 'politics',
 'politics',
 'politics',
 'entertainment',
 'entertainment',
 'tech',
 'politics',
 'sport',
 'politics',

In [71]:
pred_list = getPrediction(sentences_list)

INFO:tensorflow:Writing example 0 of 445


INFO:tensorflow:Writing example 0 of 445


INFO:tensorflow:*** Example ***


INFO:tensorflow:*** Example ***


INFO:tensorflow:guid: 


...

INFO:tensorflow:Restoring parameters from ./output_based_on_private_model/bbc/model.ckpt-556


INFO:tensorflow:Running local_init_op.


INFO:tensorflow:Running local_init_op.


INFO:tensorflow:Done running local_init_op.


INFO:tensorflow:Done running local_init_op.


In [72]:
len(pred_list)

445

In [73]:
pred_list

[('hobbit picture  four years away  lord of the rings director peter jackson has said that it will be up to four years before he starts work on a film version of the hobbit.  the oscar winner said on a visit to sydney there was a  desire  to make it  but not before lengthy negotiations.  i think it s gonna be a lot of lawyers sitting in a room trying to thrash out a deal before it will ever happen   said the new zealander. the rights to jrr tolkien s book are split between two major film studios. jackson  who is currently filming a remake of hollywood classic king kong  said he thought that the sale of mgm studios to the sony corporation would cast further uncertainty on the project. the 43-year-old was in the australian city to visit a lord of the rings exhibition  which has attracted 140 000 visitors since it opened in december.  the film-maker recently sued film company new line cinema for undisclosed damages over alleged withheld profits and lost revenue from the first part of the 

limit_output extension: Maximum message size of 100000 exceeded with 1095517 characters

In [84]:
count = 0
wrong_index_list = []

In [85]:
for i in range(len(label_list)):
    if pred_list[i][3] == label_list[i]:
        count += 1
    else:
        wrong_index_list.append(i)

In [86]:
count

437

In [87]:
wrong_index_list

[29, 153, 278, 290, 292, 327, 403, 421]

In [89]:
for i in wrong_index_list:
    print(pred_list[i][1],pred_list[i][3])
    print(label_list[i])

[2.8299203e-04 7.4471376e-05 2.6795189e-04 ... 2.5635704e-06 1.6707902e-06
 1.5673857e-06] politics
business
[9.9222106e-01 2.1787777e-03 2.4587207e-04 ... 3.1907182e-06 1.9897223e-06
 1.4694775e-06] tech
entertainment
[9.9074239e-01 3.5093157e-03 2.4908755e-04 ... 3.4512168e-06 2.1912283e-06
 1.6739731e-06] tech
entertainment
[9.9209714e-01 2.0372153e-03 2.4601843e-04 ... 3.0164433e-06 1.9516806e-06
 1.4926341e-06] tech
business
[3.4128563e-04 9.9238074e-05 2.6661201e-04 ... 1.6293405e-06 1.4105582e-06
 2.0499401e-06] business
politics
[9.9129522e-01 2.3980618e-03 2.5787676e-04 ... 3.2859991e-06 2.0351442e-06
 1.5659091e-06] tech
politics
[3.4595803e-05 1.1230022e-04 9.9694616e-01 ... 9.0983224e-07 1.1827166e-06
 1.3515552e-06] sport
tech
[2.1589080e-04 5.9627640e-05 2.0736073e-04 ... 2.4009939e-06 1.5489435e-06
 1.4986083e-06] politics
business
